<a href="https://colab.research.google.com/github/khemsu/LangGraph_tut/blob/main/Chatbot_using_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langgraph langsmith


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 6.7 MB/s eta 0:00:00


In [2]:
!pip install langchain langchain_groq langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


## From google colab


In [3]:
from google.colab import userdata
groq_api_key=userdata.get('Groq_api_key')
langsmith=userdata.get('langsmith_api_key')


In [4]:
import os
os.environ["LANGCHAIN_API_KEY"]= langsmith
os.environ["LANGCHAIN_TRACING_V2"]= "true"
os.environ["LANGCHAIN_PROJECT"]= "CourseLanggraph"

## Importing ChatGroq

In [5]:
from langchain_groq import ChatGroq

In [6]:
llm = ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")#llm for chatbot using chatgroq
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7dfd4849d190>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7dfd269e1390>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

## Now langGraph

In [7]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [8]:
graph_builder #graph is built at 0x7a518ab69610

## Creating chatbot


In [9]:
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.


## Defining node

In [10]:
#syntax: ("name-label for node", data)
graph_builder.add_node("chatbot", chatbot)

## Adding edges

In [11]:
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
graph = graph_builder.compile()

## Flow of graph


In [12]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

User: hello
Assistant: Hello! 👋 

How can I help you today? 😊

User: What is lanGraph?
Assistant: LanGraph is a research project by Google DeepMind that introduces a **new method for creating large language models (LLMs) that are more efficient and scalable**.

Here's a breakdown of what makes LanGraph unique:

* **Graph-based representation:** LanGraph uses a graph-based representation of language instead of the traditional sequential approach. This allows it to capture long-range dependencies and relationships between words more effectively.
* **Modular architecture:** LanGraph is built with a modular architecture, allowing for efficient training and scaling. Modules can be added or removed as needed, making it adaptable to different tasks.
* **Sparse attention:** LanGraph employs sparse attention mechanisms, which significantly reduce the computational cost compared to dense attention used in traditional LLMs.

**Key benefits of LanGraph:**

* **Improved performance:** LanGraph demo